# Local Gauss Thinning


In [ ]:
import igl
import meshplot as mp
import numpy as np
from numpy import linalg as LA
from gauss_thinning import *


In [ ]:
v, f = igl.read_triangle_mesh("examples/bunny_small/input.off")
mp.plot(v, f, c=v[:, 0])


## Define a path


In [ ]:
c = np.zeros(len(f))

path = [
    1100,
    1375,
    5337,
    1046,
    1252,
    1587,
    5538,
    5076,
    3699,
    5542,
    5349,
    1290,
    1300,
    1209,
    1241,
    1302,
]

for i in path:
    c[i] = 1

mp.plot(v, f, c=c)


## Increase the thickness of the path


In [ ]:
def find_active_triangles(path, adj, B, r):
    stack = []
    flag = [-1 for _ in range(len(B))]
    flag_result = [0 for _ in range(len(B))]
    result = []

    for i in path:
        stack.append(i)
        flag[i] = i

        while len(stack) > 0:
            id = stack.pop()

            if flag_result[id] == 0:
                result.append(id)
                flag_result[id] = 1

            for j in adj[id]:
                if flag[j] != i and LA.norm(B[i] - B[j]) < r:
                    stack.append(j)
                    flag[j] = i

    return result


In [ ]:
nv = len(v)
TT = triangle_adjacency(f, nv)
B = igl.barycenter(v, f)
triangle_group = find_active_triangles(path, TT, B, 0.05)

c = np.zeros(len(f))
for i in triangle_group:
    c[i] = 0.5
for i in path:
    c[i] = 1

mp.plot(v, f, c=c)


## Limit the triangles which are touched during Gauss Thinning


In [ ]:
def collect_active_neighbors(adj, B, active_triangles, N, r, nr):
    stack = []
    flag = [-1 for _ in range(len(B))]
    result = [[] for _ in range(len(B))]
    normal_cone_threshold = math.cos(nr * math.pi / 180)

    for i in active_triangles:
        stack.append(i)
        flag[i] = i

        while len(stack) > 0:
            id = stack.pop()

            result[i].append(id)

            for j in adj[id]:
                if (
                    flag[j] != i
                    and LA.norm(B[i] - B[j]) < r
                    and np.dot(N[i], N[j]) > normal_cone_threshold
                ):
                    stack.append(j)
                    flag[j] = i

    return result


def fit_active_normals(nbh, N, cosine_threshold, sigma=1):
    nf = len(nbh)
    N2 = np.zeros((nf, 3))
    angle_threshold = cosine_threshold * math.pi / 180

    for i in range(nf):
        nbi = nbh[i]

        if len(nbi) == 0:
            N2[i] = N[i]
            continue

        NN = np.zeros((len(nbi), 3))

        for k in range(len(nbi)):
            NN[k] = N[nbi[k]]

        W = -np.eye(len(nbi))

        if sigma < 10:
            for j in range(len(nbi)):
                dot = np.dot(NN[0], NN[j])
                if dot >= 1:
                    W[j][j] = 1
                elif dot < 0:
                    W[j][j] = 0
                else:
                    W[j][j] = math.exp(
                        -math.pow(math.acos(dot) / angle_threshold / sigma, 2)
                    )

        else:
            W = np.eye(len(nbi))

        _, _, frame = LA.svd(np.dot(NN.T, np.dot(W, NN)))
        left_cols = frame[0:2, :].T
        N2[i] = np.dot(np.dot(left_cols, left_cols.T), N[i].T)
        norm = LA.norm(N2[i])
        if norm > 0:
            N2[i] = N2[i] / LA.norm(N2[i])
        else:
            N2[i] = N[i]

    return N2


def local_gauss_thinning(
    V,
    F,
    path,
    brush_size=0.05,
    num_iterations=100,
    min_cone_angle=2.5,
    smooth=1e-5,
    start_angle=25,
    radius=0.1,
    sigma=2,
    render=False,
):
    cone_angle = start_angle
    eps = 1e-3

    nv = len(V)
    center(V)

    TT = triangle_adjacency(F, nv)
    C = igl.cotmatrix_entries(V, F)
    L = igl.cotmatrix(V, F)
    M = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_BARYCENTRIC).todense()
    B = igl.barycenter(V, F)
    active_triangles = find_active_triangles(path, TT, B, brush_size)

    colors = np.zeros(len(F))
    for i in active_triangles:
        colors[i] = 0.5
    for i in path:
        colors[i] = 1

    if smooth:
        A = -L + smooth * np.dot(L.T, L) + eps * M
    else:
        A = -L + eps * M

    V_dash = np.copy(V)

    if render:
        mp.plot(V_dash, F, c=colors, return_plot=True)
        plot = mp.plot(V_dash, F, c=colors, return_plot=True)

    for i in range(num_iterations):
        N = igl.per_face_normals(V_dash, F, np.array([1, 0, 0], dtype=float))
        B = igl.barycenter(V_dash, F)

        nbhs = collect_active_neighbors(TT, B, active_triangles, N, radius, cone_angle)
        if cone_angle > min_cone_angle:
            cone_angle *= 0.95

        N2 = fit_active_normals(nbhs, N, cone_angle, sigma)
        rot = find_rotations(N, N2)
        b = assemble_RHS(C, V_dash, F, rot)

        V_dash = np.asarray(LA.solve(A, eps * np.dot(M, V_dash) - b))

        if render:
            plot.update_object(vertices=V_dash)

        if i % 10 == 0:
            print(f"Finished iteration {i}.")

    if render:
        return V_dash, plot
    return V_dash


In [ ]:
v_dash = local_gauss_thinning(
    v, f, path, brush_size=0.04, num_iterations=100, min_cone_angle=5, render=True
)
